# Training a word embedding model from scratch

| Author | Last update |
|:------ |:----------- |
| Hauke Licht (https://github.com/haukelicht) | 2023-09-26 |

This notebook illustrates how to use `gensim` to train a word2vec model from scratch on original data.

## Setup

In [2]:
import os
import gensim
import numpy as np
import pandas as pd
import swifter # <== `pip install swifter` (if not yet done)

In [3]:
data_path = os.path.join('..', 'data', 'corpora', 'gbr_commons')
os.makedirs(data_path, exist_ok=True)

## Load the data

I have prepared a corpus of sentence-splitted speeches from the UK *House of Commons* (lower house chamber of the parliament).
The file is too big to be uploaded on Github.
So you need to download it if you have not yet done so:

In [4]:
fp = os.path.join(data_path, 'gbr_commons_speech_sentences_tokenized.tsv.gzip')
if not os.path.exists(fp):
    print('downloading the corpus ... might take 3-10 minutes (depending on your internet connection)')
    url = 'https://www.dropbox.com/scl/fi/wkxj7k2uiy0935dmbjp34/gbr_commons_speech_sentences_tokenized.tsv.gzip?rlkey=urjdpz0vgbymllzugzsh0m2ld&dl=1'
    corp = pd.read_csv(url, sep='\t', compression='gzip')
    corp.to_csv(fp, sep='\t', compression='gzip', index=False)
    # keep only the first 100K sentences in the corpus
    corp = corp.iloc[:100_000]
else:
    # load only the first 100K sentences in the corpus
    corp = pd.read_csv(fp, sep='\t', compression='gzip', nrows=100_000)

corp = corp[~corp.text_tokenized.isna()]

downloading the corpus ... might take 3-10 minutes (depending on your internet connection)


In [14]:
corp.head()

,text_id,text_tokenized
0,uk.org.publicwhip/debate/1970-01-19a.1.5_0_0,On a point of order
1,uk.org.publicwhip/debate/1970-01-19a.1.5_0_1,Is it true Mr Speaker that your Private Secret...
2,uk.org.publicwhip/debate/1970-01-19a.1.5_0_2,If that be true then it is a matter of deep re...
3,uk.org.publicwhip/debate/1970-01-19a.1.5_0_3,Will you Mr Speaker be kind enough to convey o...
4,uk.org.publicwhip/debate/1970-01-19a.1.7_0_0,On behalf of hon and right hon Members on this...


The data file maps sentences to sentence IDs.
The sentences have been preprocessed and tokenized into words and then concatenated with white spaces.
So to get a sentences words, we can split at the white space:

In [9]:
corp.text_tokenized[1].split(' ')

['Is',
 'it',
 'true',
 'Mr',
 'Speaker',
 'that',
 'your',
 'Private',
 'Secretary',
 'Sir',
 'Francis',
 'Reid',
 'died',
 'suddenly',
 'over',
 'the',
 'weekend']

## How to train (illustration)

To train a word2vec model on a corpus, we need to

1. create a new model instance `model = gensim.models.Word2Vec(...)`. Here we pass all model *hyper-parameters* like the embedding dimension and window size, the algorithm we want to use (Skip-gram or CBOW), etc.
2. build the model's vocabulary on our corpus `model.build_vocab(...)`
3. train the model by calling `model.train(...)`

### Train on a corpus (in memory)

All you need as **input** for these steps is a list of sentences split into tokens ("words").
That is, you input data should be prepared as follows:

```python
[
    ['Words', 'in', 'sentence', 'one'],
    ['Some', 'more', 'words', 'in', 'sentence', 'two'],
    ...
]
```

Let's split all our German *Bundestag* speech sentences into words:


In [17]:
# split pre-tokeniezd sentences into words
sentences = corp.text_tokenized.swifter.apply(lambda x: x.split(' ')).to_list()

Pandas Apply:   0%|          | 0/99997 [00:00<?, ?it/s]

In [18]:
# print first 4 sentences
sentences[:4]

[['On', 'a', 'point', 'of', 'order'],
 ['Is',
  'it',
  'true',
  'Mr',
  'Speaker',
  'that',
  'your',
  'Private',
  'Secretary',
  'Sir',
  'Francis',
  'Reid',
  'died',
  'suddenly',
  'over',
  'the',
  'weekend'],
 ['If',
  'that',
  'be',
  'true',
  'then',
  'it',
  'is',
  'a',
  'matter',
  'of',
  'deep',
  'regret',
  'to',
  'all',
  'Members',
  'on',
  'both',
  'sides',
  'of',
  'the',
  'House',
  'who',
  'shared',
  'deep',
  'friendships',
  'with',
  'Sir',
  'Francis'],
 ['Will',
  'you',
  'Mr',
  'Speaker',
  'be',
  'kind',
  'enough',
  'to',
  'convey',
  'our',
  'deep',
  'sympathy',
  'to',
  'the',
  'members',
  'of',
  'his',
  'family']]

Now we are ready to create a `Word2Vec` model instance.
The following model hyper-parameters are relevant for our purposes:

- `vector_size`: the number of dimensions $d$ of th word embedding matrix
- `window`: the number of context words to use for modeling 
- `min_count`: the minimum number of times a word needs to occur in the corpus to be included in the vocabulary
- `sg`: set to 1 to use the Skip-Gram algorithm, otherwise uses CBOW algorithm
- `hs`: set to 1 to use the hierarchical softmax
- `negative`: number of negative samples to use per focus word when computing the loss
- `epochs`: number of times to iterate over corpus (all sentences)
- `workers`: number of CPU cores to use for training parallelization

In [19]:
import gensim

# create a new model instance
model = gensim.models.Word2Vec(
    vector_size=10, # <= super low dimensionality for testing
    window=5,
    min_count=5,
    sg=1,
    hs=1,
    negative=5,
    epochs=5, # <= set very low for testing
    workers=10
)

Next, we build the vocabulary:

In [20]:
# build the vocabulary from the list of sentences
model.build_vocab(sentences)

Now we can train the model:

In [21]:
# train the model on the list of sentences
model.train(sentences, total_examples=model.corpus_count, epochs=model.epochs)

(7101192, 10061760)

Let's compute similarities on a number of hand-picked word pairs to see whether the model has learned sensible word vectors.

*Note:* This is more of a "face validity" check and no full-fledged validation

In [22]:
# evaluate
pairs = [
    ('Sir', 'Madan'),
    ('Pound', 'Euro'),
    ('Pound', 'Dollar'),
    ('Government', 'law'),
    ('Government', 'bill'),
    ('Government', 'propoasal'),
    ('Opposition', 'law'),
    ('Opposition', 'bill'),
    ('Opposition', 'proposal'),
    ('bill', 'proposal'),
    ('Labour', 'government'),
    ('Labour', 'opposition'),
    ('Conservatives', 'government'),
    ('Conservatives', 'opposition'),
]
for pair in pairs:
    if all(p in model.wv.key_to_index for p in pair):
        print(f'"{pair[0]}" - "{pair[1]}": {model.wv.similarity(pair[0], pair[1]):.3f}')

"Government" - "law": 0.408
"Government" - "bill": 0.277
"Opposition" - "law": 0.350
"Opposition" - "bill": 0.137
"Opposition" - "proposal": 0.747
"bill" - "proposal": 0.362
"Labour" - "government": 0.479
"Labour" - "opposition": 0.654
"Conservatives" - "government": 0.357
"Conservatives" - "opposition": 0.620


### Train with a corpus read from disk

The above code works just fine.
But it has a computational bottleneck!
To iterate over the list of sentences, we need to have them all loaded into our working memory (RAM).
If our corpus is very large -- which it should be to enable learning of reliable word embeddings --, this might be too burdensome for your computer.

So instead of loading the entire corpus in the RAW, we can iteratively read batches of sentences from a data file that exists somewhere on the hard drive ("disk") of your computer.
For this, we need an "iterable" class that implements "yields" sentences in our corpus one at a time.

In the next code cell, we define such a class.
This class reads the sentences from a (ZIP-ed) CSV file and splits sentences at white spaces before yielding them.


In [24]:
import gzip
from tqdm.auto import tqdm

# get corpus sentences
class SentenceCorpus(object):
    """Iterable class for reading a corpus from a file.
    
    Parameters
    ----------
    filepath : str
        Path to the corpus file.
    sep : str, optional
        Separator for splitting the lines into tokens. Default: ','
    compressed : bool, optional
        Whether the file is compressed. Default: False
    skip : int, optional
        Number of lines to skip at the beginning of the file. Default: 1
    nrows : int, optional
        Number of lines to read from the file. Default: None (read all lines)
    
    Yields
    ------
    list of str

    """
    # note: I've already added a lot of functionality here (e.g. reading from compressed files, skip rows, limit the number of rows to read).
    #       Hence, the actual code is a bit more complex than what you'd need in some use cases.
    def __init__(self, filepath, sep='\t', compressed=False, skip=1, nrows=None):
        self.filepath = filepath
        self.sep = sep
        self.compressed = compressed
        self.skip = skip if skip else 0
        self.nrows = nrows if nrows else np.inf
        if skip and skip > 0:
            self.nrows += skip
        with gzip.open(self.filepath,'rt') if self.compressed else open(self.filepath, 'r') as file:
            corpus_size = len(file.readlines())
        if nrows is not None and nrows < corpus_size:
            corpus_size = nrows
        else:
            nrows = corpus_size
        self.corpus_size = nrows
    
    def __len__(self):
        return self.corpus_size
    
    def __iter__(self):
        with gzip.open(self.filepath,'rt') if self.compressed else open(self.filepath, 'r') as file:
            max_ = self.nrows if self.skip <= 0 else self.nrows+self.skip
            for i, line in enumerate(file):
                if i <= self.skip:
                    continue
                if max_ and i >= max_:
                    break
                yield line.strip().split(self.sep)[1].split(' ')

Let's read the some 100,000 sentences from our corpus as above:

In [25]:
fp = os.path.join(data_path, 'gbr_commons_speech_sentences_tokenized.tsv.gzip')
sentences = SentenceCorpus(filepath=fp, compressed=True, nrows=100_000)
# note: loading it takes time because it's an iterable

In [26]:
len(sentences)

100000

In [27]:
model = gensim.models.Word2Vec(
    vector_size=10, # <= super low dimensionality for testing
    window=5,
    min_count=5,
    sg=1,
    hs=1,
    negative=5,
    epochs=5, # <= set very low for testing
    workers=10
)

In [28]:
model.build_vocab(sentences)

In [29]:
model.train(sentences, total_examples=model.corpus_count, epochs=model.epochs)

(7102431, 10061815)

Let's again compute similarities on our hand-picked word pairs:

In [30]:
for pair in pairs:
    if all(p in model.wv.key_to_index for p in pair):
        print(f'"{pair[0]}" - "{pair[1]}": {model.wv.similarity(pair[0], pair[1]):.3f}')

"Government" - "law": 0.382
"Government" - "bill": 0.349
"Opposition" - "law": 0.307
"Opposition" - "bill": 0.235
"Opposition" - "proposal": 0.737
"bill" - "proposal": 0.331
"Labour" - "government": 0.495
"Labour" - "opposition": 0.729
"Conservatives" - "government": 0.343
"Conservatives" - "opposition": 0.713


## Train, for real

In [31]:
fp = os.path.join(data_path, 'gbr_commons_speech_sentences_tokenized.tsv.gzip')
sentences = SentenceCorpus(filepath=fp, compressed=True)
len(sentences)

26290960

In [32]:
model = gensim.models.Word2Vec(
    vector_size=100, # <= low dimensionality for testing
    window=5,
    min_count=25, # <= words should appear at least 25 times in the corpus
    sg=1,
    hs=1,
    negative=5,
    epochs=10, # <= set very low for testing
    workers=10,
    sorted_vocab=1,
)

In [33]:
model.build_vocab(sentences) # <== takes long because corpus is large 

Because training is going to take a while, we'll add a "callback" function that prints the loss after each epoch

*source:* https://stackoverflow.com/a/54891714

In [34]:
import time
import gensim

class callback(gensim.models.callbacks.CallbackAny2Vec):
    '''Callback to print loss after each epoch.'''

    def __init__(self):
        self.epoch = 0
        # record the start time of the training
    
    def on_epoch_begin(self, model):
        print(f'Epoch #{self.epoch} start ...', end=' ')
        self.ts = time.time()

    def on_epoch_end(self, model):
        # compute the time delta of the epoch relative to self.ts
        delta = time.time() - self.ts
        # print the delta time and loss after each epoch
        loss = model.get_latest_training_loss()
        print(f'Epoch #{self.epoch} took {int(delta // 60)}:{int(delta % 60)}m; loss = {loss}')
        self.epoch += 1

In [35]:
model.train(
    sentences, 
    total_examples=model.corpus_count, 
    epochs=model.epochs,
    compute_loss=True, 
    callbacks=[callback()]
)

Epoch #0 start ... Epoch #0 took 10:58m; loss = 77135600.0
Epoch #1 start ... Epoch #1 took 11:45m; loss = 90628424.0
Epoch #2 start ... Epoch #2 took 11:19m; loss = 103634312.0
Epoch #3 start ... Epoch #3 took 11:21m; loss = 116353824.0
Epoch #4 start ... Epoch #4 took 11:22m; loss = 128649688.0
Epoch #5 start ... Epoch #5 took 11:23m; loss = 134217728.0
Epoch #6 start ... Epoch #6 took 11:41m; loss = 134217728.0
Epoch #7 start ... Epoch #7 took 11:28m; loss = 134217728.0
Epoch #8 start ... Epoch #8 took 11:43m; loss = 134217728.0
Epoch #9 start ... Epoch #9 took 10:19m; loss = 134217728.0


(3791431089, 5196179950)

In [37]:
# Let's again compute similarities on our hand-picked word pairs:
for pair in pairs:
    if all(p in model.wv.key_to_index for p in pair):
        print(f'"{pair[0]}" - "{pair[1]}": {model.wv.similarity(pair[0], pair[1]):.3f}')

"Pound" - "Euro": 0.249
"Pound" - "Dollar": 0.188
"Government" - "law": 0.360
"Government" - "bill": 0.223
"Opposition" - "law": 0.126
"Opposition" - "bill": 0.091
"Opposition" - "proposal": 0.312
"bill" - "proposal": 0.349
"Labour" - "government": 0.618
"Labour" - "opposition": 0.533
"Conservatives" - "government": 0.545
"Conservatives" - "opposition": 0.515


## Saving the model and it's vectors

In [43]:
# store the model's word vectors in a file
models_path = data_path.replace('corpora', 'models')
print(models_path)
fp = os.path.join(models_path, 'gbr_commons_word2vec_w5_d100')

../../../data/models/gbr_commons


With a trained model at hand, you have two options for saving it for re-use.
You can only save the word vectors (as `KeyedVectors`).
Or you can save the full model (which inlcudes the word vectors but has more capabilities).

The following table compares the **pros and cons** of these approaches ([source](https://radimrehurek.com/gensim/models/keyedvectors.html#how-to-obtain-word-vectors)):

<table class="docutils align-default">
<colgroup>
<col style="width: 24%">
<col style="width: 12%">
<col style="width: 11%">
<col style="width: 54%">
</colgroup>
<tbody>
<tr class="row-odd">
<td><p><em>capability</em></p></td>
<td><p><em>KeyedVectors</em></p></td>
<td><p><em>full model</em></p></td>
<td><p><em>note</em></p></td>
</tr>
<tr class="row-even"><td><p>continue training vectors</p></td>
<td><p>❌</p></td>
<td><p>✅</p></td>
<td><p>You need the full model to train or update vectors.</p></td>
</tr>
<tr class="row-odd"><td><p>smaller objects</p></td>
<td><p>✅</p></td>
<td><p>❌</p></td>
<td><p>KeyedVectors are smaller and need less RAM, because they
don’t need to store the model state that enables training.</p></td>
</tr>
<tr class="row-even"><td><p>save/load from native
fasttext/word2vec format</p></td>
<td><p>✅</p></td>
<td><p>❌</p></td>
<td><p>Vectors exported by the Facebook and Google tools
do not support further training, but you can still load
them into KeyedVectors.</p></td>
</tr>
<tr class="row-odd"><td><p>append new vectors</p></td>
<td><p>✅</p></td>
<td><p>✅</p></td>
<td><p>Add new-vector entries to the mapping dynamically.</p></td>
</tr>
<tr class="row-even"><td><p>concurrency</p></td>
<td><p>✅</p></td>
<td><p>✅</p></td>
<td><p>Thread-safe, allows concurrent vector queries.</p></td>
</tr>
<tr class="row-odd"><td><p>shared RAM</p></td>
<td><p>✅</p></td>
<td><p>✅</p></td>
<td><p>Multiple processes can re-use the same data, keeping only
a single copy in RAM using
<a class="reference external" href="https://en.wikipedia.org/wiki/Mmap">mmap</a>.</p></td>
</tr>
<tr class="row-even"><td><p>fast load</p></td>
<td><p>✅</p></td>
<td><p>✅</p></td>
<td><p>Supports <a class="reference external" href="https://en.wikipedia.org/wiki/Mmap">mmap</a>
to load data from disk instantaneously.</p></td>
</tr>
</tbody>
</table>

#### Saving the full model

Just call the save `method()` on your word2vec `mode` object

**_Note:_** It's customary to name the file you write the model to with the file extension '.model' (we will use another one when we just save the word vectors) 


In [44]:
model.save(fp+'.model')

It's then easy to reload the model:

In [48]:
from gensim.models import Word2Vec
tmp = Word2Vec.load(fp+'.model')
type(tmp), type(tmp.wv)

(gensim.models.word2vec.Word2Vec, gensim.models.keyedvectors.KeyedVectors)

#### Saving only the word vectors

Call the save `method()` on your word2vec `mode`'s `wv` attribute (a `KeyedVectors` object recording the model's word vectors):

**_Note:_** It's customary to name the file you write the model to with the file extension '.kv' 🥝


In [51]:
model.wv.save(fp+'.kv')

In [52]:
from gensim.models import KeyedVectors
tmp = KeyedVectors.load(fp+'.kv')
type(tmp)

gensim.models.keyedvectors.KeyedVectors

#### Save in other standard formats

Since Word2Vec has been implemented in many other languages (e.g., C and R), there is a standardized file format for storing word vectors.
Saving word embeddings in this format allows "interoperatbility" between languages. 
So for example, you could train your word embedding model in Python but compute with the embeddings in R.

In [54]:
model.wv.save_word2vec_format(fp+'.vectors')

In [60]:
# inspect the file content (first two lines only) in the Terminal
!head -n 3 ../data/models/gbr_commons/gbr_commons_word2vec_w5_d100.vectors
# note: on Windows, use 'more /n 3' instead of 'head -n 3' and backward instead of forward slashes

79338 100
the -0.115997605 -0.14884225 -0.12895775 0.09393551 0.097044945 -0.30353922 0.19999993 0.07260028 0.022534432 0.27119234 -0.376409 0.021482823 -0.026498402 -0.19890901 0.23564409 0.14851832 0.22597332 0.018340524 -0.13723059 0.14085543 -0.11470687 0.06691807 -0.056911133 0.050308105 0.060546733 0.055016436 -0.2705088 -0.037383683 0.28965232 -0.055596534 0.21220706 -0.18371496 0.13106298 -0.07839364 -0.045897573 -0.13771228 -0.14794247 0.13426219 -0.18550311 0.035354722 0.2651762 0.067005426 -0.002249805 0.20813881 -0.08645588 0.20668018 -0.05650302 0.08491766 0.32132623 -0.13516597 -0.084183134 0.11043672 0.18633321 -0.45208684 0.06992645 -0.13670206 0.15804847 0.010294967 -0.19854228 -0.1443379 0.14387698 0.3035268 0.032096878 -0.1608676 -0.0804171 -0.027760101 -0.018719688 -0.2336726 0.0868613 -0.13737299 0.22800523 -0.019262169 -0.07829984 -0.25567237 0.13903368 -0.0075952252 -0.11072661 -0.33697185 -0.06383023 -0.1913333 -0.17815098 0.09914854 -0.113653354 -0.121188246 0.